# FABLIB Tools


## Import the FABlib Library
Chose one of two working enviroments to import FABlib.
- fabric-testbed
- local host

### Environment: Fabric-Testbed

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import json

try:
    fablib = fablib_manager()
                     
    fablib.show_config();
except Exception as e:
    print(f"Exception: {e}")

### Environment: Local Host

Get Fabric configuration from FABFED credential file
- Set the path to fabfed credential file
- Read the fabric configuration from fabfed crential file
- Set the path to ssh_config the same as that of the bastion_ssh_key
- Generate fabric_rc
- Import Fablib

In [ ]:
# Path to your YAML file
credential_file_path = 'fabfed_config/fabfed_credentials.yml'

In [ ]:
import yaml

# Read the YAML file
with open(credential_file_path, 'r') as file:
    try:
        yaml_data = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

fabric_config = yaml_data['fabric']

def print_dict(dictionary, indent=0):
    for key, value in dictionary.items():
        if isinstance(value, dict):
            print('  ' * indent + str(key) + ':')
            print_dict(value, indent + 1)
        else:
            print('  ' * indent + str(key) + ': ' + str(value))
            
print_dict(fabric_config)

In [ ]:
import os

home_directory = os.path.expanduser("~")
print("Home directory:", home_directory)

bastion_ssh_config_file = os.path.dirname(fabric_config['bastion-key-location']) + "/ssh_config"
print(f"ssh_config path: {bastion_ssh_config_file}")

In [ ]:
# Data to write to the file
data = f"""
export FABRIC_CREDMGR_HOST=cm.fabric-testbed.net
export FABRIC_ORCHESTRATOR_HOST=orchestrator.fabric-testbed.net
export FABRIC_PROJECT_ID={fabric_config['project_id']}
export FABRIC_TOKEN_LOCATION={fabric_config['token-location']}

export FABRIC_BASTION_HOST=bastion.fabric-testbed.net
export FABRIC_BASTION_USERNAME={fabric_config['bastion-user-name']}

export FABRIC_BASTION_KEY_LOCATION={fabric_config['bastion-key-location']}
#export FABRIC_BASTION_KEY_PASSPHRASE=

export FABRIC_BASTION_SSH_CONFIG_FILE={bastion_ssh_config_file}

export FABRIC_SLICE_PRIVATE_KEY_FILE={fabric_config['slice-private-key-location']}
export FABRIC_SLICE_PUBLIC_KEY_FILE={fabric_config['slice-public-key-location']} 
#export FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE=

export FABRIC_LOG_FILE={home_directory}/fablib.log
export FABRIC_LOG_LEVEL=INFO 

export FABRIC_AVOID=''

export FABRIC_SSH_COMMAND_LINE="ssh -i {{ _self_.private_ssh_key_file }} -F {bastion_ssh_config_file} {{ _self_.username }}@{{ _self_.management_ip }}"
"""

# Specify the file path
fabric_rc_file = "./fabric_config/fabric_rc"

# Delete if it exists
if os.path.exists(fabric_rc_file):
    # Delete the file
    os.remove(fabric_rc_file)
    
# Open the file in write mode
with open(fabric_rc_file, "w") as file:
    # Write the data to the file
    file.write(data)

print("Data has been written to", fabric_rc_file)


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import json

try:
    fablib = fablib_manager(fabric_rc=fabric_rc_file)
                     
    fablib.show_config();
except Exception as e:
    print(f"Exception: {e}")

## Show Slice Information

### List all slices

In [ ]:
fablib.list_slices()

print("Complete!")

### Show specific slice

Observe the specific slice's attributes

In [ ]:
slice_name = "native-gcp-demo"

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")